## Transfer Learning

### Install Libraries

In [1]:
# Importing sys to check the current Python executable
import sys
print(sys.executable)

# Import necessary libraries and install if they are not already installed
import subprocess

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of required packages
required_packages = [
    'numpy', 'pandas', 'opencv-python', 'Pillow', 'torch', 'torchvision',
    'scikit-learn', 'jupyter'
]

# Install the required packages
for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        install(package)

# Now you can safely import the packages
import numpy as np
import pandas as pd
import os
import cv2
from PIL import Image
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score

print("All packages are installed and imported successfully!")


c:\Users\ASUS\Desktop\SPRING_24\Learning_From_Data\project\code\transfer_learning\venv\Scripts\python.exe
All packages are installed and imported successfully!


### Load the training data